In [ ]:
%load_ext autoreload
%autoreload

# Download dataset
Download nad unpack SVHN dataset to datasets/SVHN/[train|test] folder

In [ ]:
from keras_detection.datasets.svhn.read_svhm_map import load_dataset

train_raw_dataset, num_train_examples = load_dataset('datasets/SVHN/train')
test_raw_dataset, num_test_examples = load_dataset('datasets/SVHN/test')

In [ ]:
num_train_examples, num_test_examples

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
keras = tf.keras

In [ ]:
# disable tensorflow duplicated logs
from tensorflow.python.platform.tf_logging import _logger
_logger.handlers.pop(0)

In [ ]:
train_raw_dataset

# Define model input and dataset parametres

In [ ]:
num_classes = 10
image_dim = 128
batch_size = 8
num_epochs = -1
num_parallel_calls = 8
num_test_steps = num_test_examples // batch_size

In [ ]:
from keras_detection.datasets.datasets_ops import prepare_dataset

In [ ]:
train_dataset = prepare_dataset(
    dataset=train_raw_dataset, 
    augmentation_fn=None,
    model_image_size=(image_dim, image_dim),
    batch_size=batch_size,
    num_epochs=num_epochs,
    shuffle_buffer_size=512,
    prefetch_buffer_size=2,
    num_parallel_calls=num_parallel_calls
)

test_dataset = prepare_dataset(
    dataset=test_raw_dataset, 
    augmentation_fn=None,
    model_image_size=(image_dim, image_dim),
    batch_size=batch_size,
    num_epochs=num_epochs,
    shuffle_buffer_size=None,
    prefetch_buffer_size=2,
    num_parallel_calls=num_parallel_calls
)

# Define model

In [ ]:
import keras_detection.backbones.mobilenetv2_customized as mobilenet
import keras_detection.tasks.standard_tasks as standard_tasks
import keras_detection.models.fpn_builder as fpn_builder
from keras_detection.utils import plotting

In [ ]:
backbone = mobilenet.MobileNetV2Backbone(
    input_shape=[image_dim, image_dim, 3],
    alpha=1.0, min_fm_size=10, weights='imagenet'
)

tasks = [
    # predicts objectnes score for each anchor
    standard_tasks.get_objectness_task(label_smoothing=0.02, obj_class="center_ignore_margin"),
    # predicts [height, with, y, x ] location of the box
    standard_tasks.get_box_shape_task("box_shape"),
    # predicts [num_classes] for each anchor
    standard_tasks.get_multiclass_task(num_classes, fl_gamma=0.0, label_smoothing=0, activation='softmax'),
]

In [ ]:
builder = fpn_builder.FPNBuilder(backbone = backbone, tasks = tasks)
model = builder.build(batch_size=None)

In [ ]:
model.summary()

# Train model

In [ ]:
prepared_train_dataset = train_dataset.map(builder.get_build_training_targets_fn())

## Check targets

In [ ]:
features, labels = next(iter(prepared_train_dataset))

In [ ]:
fm = "fm16x16"

targets = [labels[n][..., :-1] for n in builder.get_outputs_names()]
targets = builder.predictions_to_dict(targets, postprocess=True)

idx = 6
target=dict(
    objectness=targets[f'{fm}/objectness'][idx],
    boxes_shape_map=targets[f'{fm}/box_shape'][idx],
    classes_map=targets[f'{fm}/classes'][idx],
)

render = plotting.draw_compares(    
    target=target,
    predicted=None,
    all_targets=True,
    draw_fns=[
        plotting.draw_boxes, 
        plotting.draw_classes_map,
        plotting.draw_objectness_map,
        plotting.draw_classes_max_score_map,
    ],
    image=features['image'][idx] / 255,
    score_threshold=0.2, 
)
render

## Run training model

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.005, beta_2=0.99)
model.compile(optimizer, **builder.get_model_compile_args())
model.fit(prepared_train_dataset, epochs=1, steps_per_epoch=200)

In [ ]:
import keras_detection.models.utils as kd_utils

In [ ]:
kd_utils.freeze_model_bn_layers(model, trainable=False)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_2=0.99)
model.compile(optimizer, **builder.get_model_compile_args())
model.fit(prepared_train_dataset, epochs=1, steps_per_epoch=100)

In [ ]:
kd_utils.freeze_model_bn_layers(model, trainable=True)
model.save_weights("models/non-quantized-model.h5")

In [ ]:
model.load_weights("models/non-quantized-model.h5")

# Test trained model

In [ ]:
builder.evaluate_model(model, test_dataset, num_test_steps)

In [ ]:
prepared_test_dataset = test_dataset.map(builder.get_build_training_targets_fn())
dataset_iterator = iter(prepared_test_dataset)

In [ ]:
features, labels = next(dataset_iterator)
predictions = model.predict(features)
predictions = builder.predictions_to_dict(predictions, postprocess=True)

In [ ]:
builder.get_outputs_names()

In [ ]:
fm = "fm16x16"
targets = [labels[n][..., :-1] for n in builder.get_outputs_names()]
targets = builder.predictions_to_dict(targets, postprocess=True)

In [ ]:
idx = 0
target=dict(
    objectness=targets[f'{fm}/objectness'][idx],
    boxes_shape_map=targets[f'{fm}/box_shape'][idx],
    classes_map=targets[f'{fm}/classes'][idx],
)

predicted=dict(
    objectness=predictions[f'{fm}/objectness'][idx],
    boxes_shape_map=predictions[f'{fm}/box_shape'][idx],
    classes_map=predictions[f'{fm}/classes'][idx],
)

render = plotting.draw_compares(    
    target=target,
    predicted=predicted,
    draw_fns=[
        plotting.draw_boxes, 
        plotting.draw_classes_map,
        plotting.draw_objectness_map,
        plotting.draw_classes_max_score_map,
    ],
    image=features['image'][idx] / 255,
    score_threshold=0.2,
)
render

In [ ]:
quantized_model = builder.build_quantized(
    batch_size=None, 
    non_quantized_model_weights="models/non-quantized-model.h5"
)

In [ ]:
quantized_model.summary()

In [ ]:
builder.evaluate_model(quantized_model, test_dataset, num_test_steps)

# Train quantized model

In [ ]:
prepared_train_dataset = train_dataset.map(builder.get_build_training_targets_fn())

In [ ]:
kd_utils.freeze_model_bn_layers(quantized_model, trainable=False)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_2=0.99)
quantized_model.compile(optimizer, **builder.get_model_compile_args())
quantized_model.fit(prepared_train_dataset, epochs=1, steps_per_epoch=100)

In [ ]:
kd_utils.freeze_model_bn_layers(quantized_model, trainable=True)
quantized_model.save_weights("models/quantized-model.h5")

In [ ]:
quantized_model.summary()

# Export model to tflite

In [ ]:
exported_model, tflite_models_paths = builder.convert_to_tflite(
    model, 
    save_path="models/quantized-model.tflite", 
    export_batch_size = 1,
    raw_dataset = test_raw_dataset,
    num_dataset_samples = 16,
    num_test_steps = 8,
    merge_feature_maps=True,
    postprocess_outputs=True,
    convert_quantized_model=True
)

# Create box-predictor from converted model

In [ ]:
box_detector =  builder.as_box_detector("models/quantized-model.h5", is_quantized=True)

In [ ]:
predictions = box_detector.predict(features['image'])

In [ ]:
predictions[0].draw(image=features['image'][0].numpy())